<a href="https://colab.research.google.com/github/kimdesok/Text-classification/blob/main/Spam_Classification_Keras_Dense_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
print(tf.__version__)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#DIR_PATH = r"/home/ubuntu/Kaggle2/Data"
DIR_PATH = r"/content/spam.csv"
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2.11.0


In [6]:
# Read the data 
df = pd.read_csv(DIR_PATH, encoding="latin-1")
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
# Shuffled the data again to be sure
df_shuffled = df.sample(frac=1, random_state=42)
df_shuffled.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
3245,ham,"Funny fact Nobody teaches volcanoes 2 erupt, t...",NaN,NaN,NaN
944,ham,I sent my scores to sophas and i had to do sec...,NaN,NaN,NaN
1044,spam,We know someone who you know that fancies you....,NaN,NaN,NaN
2484,ham,Only if you promise your getting out as SOON a...,NaN,NaN,NaN
812,spam,Congratulations ur awarded either å£500 of CD ...,NaN,NaN,NaN


In [ ]:
df_shuffled["text"] = df_shuffled["v2"]
df_shuffled['target'] = df_shuffled['v1'].map({'ham': 0, 'spam': 1})
df_shuffled.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,text,target
3245,ham,"Funny fact Nobody teaches volcanoes 2 erupt, t...",NaN,NaN,NaN,"Funny fact Nobody teaches volcanoes 2 erupt, t...",0
944,ham,I sent my scores to sophas and i had to do sec...,NaN,NaN,NaN,I sent my scores to sophas and i had to do sec...,0
1044,spam,We know someone who you know that fancies you....,NaN,NaN,NaN,We know someone who you know that fancies you....,1
2484,ham,Only if you promise your getting out as SOON a...,NaN,NaN,NaN,Only if you promise your getting out as SOON a...,0
812,spam,Congratulations ur awarded either å£500 of CD ...,NaN,NaN,NaN,Congratulations ur awarded either å£500 of CD ...,1


In [ ]:
# How many examples of each class?
df_shuffled.target.value_counts()

0    4825
1     747
Name: target, dtype: int64

In [ ]:
# How many samples?
print(f"Total samples: {len(df_shuffled)}")

Total samples: 5572


In [ ]:
# Visualize some random example
import random
rand_index = random.randint(0, len(df_shuffled)-10)
for row in df_shuffled[["text","target"]][rand_index:rand_index+10].itertuples():
    _,text,target = row
    print(f"Target: {target}", "(spam)" if target > 0 else "(not spam)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 0 (not spam)
Text:
Really good:)dhanush rocks once again:)

---

Target: 0 (not spam)
Text:
I will be outside office take all from there

---

Target: 0 (not spam)
Text:
Thanx 4 e brownie it's v nice...

---

Target: 0 (not spam)
Text:
Cool, I'll text you when I'm on the way

---

Target: 0 (not spam)
Text:
Oh you got many responsibilities.

---

Target: 1 (spam)
Text:
Hello from Orange. For 1 month's free access to games, news and sport, plus 10 free texts and 20 photo messages, reply YES. Terms apply: www.orange.co.uk/ow

---

Target: 0 (not spam)
Text:
Wow didn't think it was that common. I take it all back ur not a freak! Unless u chop it off:-)

---

Target: 0 (not spam)
Text:
Its sunny in california. The weather's just cool

---

Target: 0 (not spam)
Text:
I jus hope its true that  missin me cos i'm really missin him! You haven't done anything to feel guilty about, yet.

---

Target: 0 (not spam)
Text:
It wont b until 2.15 as trying 2 sort house out, is that ok?

---



In [ ]:
# Let's create train and test split
from sklearn.model_selection import train_test_split
X_train_sentences, X_val_sentences, y_train_labels, y_val_labels = train_test_split(df_shuffled["text"].to_numpy(),
                                                                            df_shuffled["target"].to_numpy(),
                                                                            test_size=0.2,
                                                                            random_state=42)

In [ ]:
#print(type(df_shuffled["text"]), type(df_shuffled["text"].to_numpy()))

In [ ]:
#### Check the lengths
len(X_train_sentences), len(y_train_labels), len(X_val_sentences), len(y_val_labels)

(4457, 4457, 1115, 1115)

In [ ]:
# View first 5 training and their label sentences :
X_train_sentences[:5], y_train_labels[:5]

(array(['from www.Applausestore.com MonthlySubscription@50p/msg max6/month T&CsC web age16 2stop txt stop',
        'Ranjith cal drpd Deeraj and deepak 5min hold',
        'I cant pick the phone right now. Pls send a message',
        'New Mobiles from 2004, MUST GO! Txt: NOKIA to No: 89545 & collect yours today! From ONLY å£1. www.4-tc.biz 2optout 087187262701.50gbp/mtmsg18 TXTAUCTION.',
        "Honestly i've just made a lovely cup of tea and promptly dropped my keys in it and then burnt my fingers getting them out!"],
       dtype=object),
 array([1, 0, 0, 1, 0]))

In [ ]:
# Text Vectorization
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=None,
                                                    standardize='lower_and_strip_punctuation',
                                                    split='whitespace',
                                                    ngrams=None,
                                                    output_mode='int',
                                                    output_sequence_length=None,
                                                    )

2023-02-24 21:33:18.732771: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-24 21:33:18.736370: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-24 21:33:18.740032: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-24 21:33:18.743595: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-24 21:33:18.896074: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

In [ ]:
text_vectorizer

In [ ]:
# Finding average number of words or token
round(sum([len(i.split()) for i in X_train_sentences])/len(X_train_sentences))

16

In [ ]:
max_vocab_length = 9000
max_length = 16

text_vectorizer=tf.keras.layers.TextVectorization(max_tokens=max_vocab_length,
                                                  output_mode="int",
                                                  output_sequence_length=max_length)

In [ ]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(X_train_sentences)

In [ ]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(X_train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
Wot about on wed nite I am 3 then but only til 9!      

Vectorized version:


<tf.Tensor: shape=(1, 16), dtype=int64, numpy=
array([[493,  85,  19, 903, 569,   2,  65, 199,  63,  27,  68, 436, 432,
          0,   0,   0]])>

In [ ]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] means "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 8456
Top 5 most common words: ['', '[UNK]', 'i', 'to', 'you']
Bottom 5 least common words: ['02', '0125698789', '0121', '0089my', '008704050406']


In [ ]:
# Embedding 

tf.random.set_seed(42)
embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,
                                      output_dim=128,
                                      embeddings_initializer='uniform',
                                      input_length=max_length,
                                      name="embedding")


In [ ]:
# Get a random sentence from training set
random_sentence = random.choice(X_train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embedding the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Claim a 200 shopping spree, just call 08717895698 now! Have you won! MobStoreQuiz10ppm      

Embedded version:


<tf.Tensor: shape=(1, 16, 128), dtype=float32, numpy=
array([[[ 0.0291287 ,  0.02585142,  0.0255974 , ...,  0.00572053,
          0.02781173,  0.01084658],
        [-0.03521683, -0.00981399, -0.00284491, ..., -0.04792733,
         -0.01664252, -0.03293632],
        [ 0.01842329, -0.01247717, -0.00487962, ..., -0.02920858,
          0.03259977,  0.00279276],
        ...,
        [ 0.01645621, -0.00589932, -0.01471175, ..., -0.02511839,
          0.00912381, -0.00024097],
        [ 0.01645621, -0.00589932, -0.01471175, ..., -0.02511839,
          0.00912381, -0.00024097],
        [ 0.01645621, -0.00589932, -0.01471175, ..., -0.02511839,
          0.00912381, -0.00024097]]], dtype=float32)>

In [ ]:
# Get the config of our text vectorizer
text_vectorizer.get_config()

{'name': 'text_vectorization_1',
 'trainable': True,
 'batch_input_shape': (None,),
 'dtype': 'string',
 'max_tokens': 9000,
 'standardize': 'lower_and_strip_punctuation',
 'split': 'whitespace',
 'ngrams': None,
 'output_mode': 'int',
 'output_sequence_length': 16,
 'pad_to_max_tokens': False,
 'sparse': False,
 'ragged': False,
 'vocabulary': None,
 'idf_weights': None}

In [ ]:
# Bulding Baseline Model using TF-IDF Multinomial Naive Bayes
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

baseline_model = Pipeline([
    ("tf-idf", TfidfVectorizer()), # converts words to numbers
    ("clf", MultinomialNB())
])

baseline_model.fit(X_train_sentences,
                   y_train_labels)



Pipeline(steps=[('tf-idf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [ ]:
# Evaluate baseline_model's score
baseline_score = baseline_model.score(X_val_sentences, y_val_labels)
print(f"Our baseline_score accuracy is: {baseline_score*100:.2f}%")



Our baseline_score accuracy is: 95.61%


In [ ]:
# Make predictions
baseline_preds = baseline_model.predict(X_val_sentences)
baseline_preds[:10]

array([0, 0, 1, 1, 0, 0, 0, 0, 0, 0])

In [ ]:
# Function to evaluate; accuracy, precision, recall and f1 score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    model_precision, model_recall, model_f1,_ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {"accuracy": model_accuracy,
                     "precision": model_precision,
                     "recall": model_recall,
                     "f1": model_f1}
    return model_results
    

In [ ]:
baseline_model_results = calculate_results(y_true=y_val_labels,
                                           y_pred=baseline_preds)
baseline_model_results

{'accuracy': 95.60538116591928,
 'precision': 0.958211490245064,
 'recall': 0.9560538116591928,
 'f1': 0.9528022781660086}

In [ ]:
# Model: Simple dense model
inputs = tf.keras.layers.Input(shape=(1,), dtype="string") #1 d strings
x = text_vectorizer(inputs) # turn texts into number
x = embedding(x) # create an embedding of the numerized numbers
x = tf.keras.layers.GlobalAveragePooling1D()(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
simple_dense_model = tf.keras.Model(inputs, outputs, name="simple_dense_model")

# Compile the model
simple_dense_model.compile(loss="binary_crossentropy",
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=["accuracy"])

simple_dense_model.summary()

Model: "simple_dense_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 16)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 16, 128)           1152000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,152,129
Trainable params: 1,152,

In [ ]:
# Fit the model
history_simple_dense_model = simple_dense_model.fit(X_train_sentences,
                                                    y_train_labels,
                                                    epochs=10,
                                                    validation_data=(X_val_sentences, y_val_labels))

Epoch 1/10
140/140 [==============================] - 1s 6ms/step - loss: 0.0405 - accuracy: 0.9901 - val_loss: 0.0778 - val_accuracy: 0.9713
Epoch 2/10
140/140 [==============================] - 1s 6ms/step - loss: 0.0325 - accuracy: 0.9906 - val_loss: 0.0738 - val_accuracy: 0.9749
Epoch 3/10
140/140 [==============================] - 1s 6ms/step - loss: 0.0263 - accuracy: 0.9935 - val_loss: 0.0715 - val_accuracy: 0.9758
Epoch 4/10
140/140 [==============================] - 1s 6ms/step - loss: 0.0215 - accuracy: 0.9946 - val_loss: 0.0700 - val_accuracy: 0.9749
Epoch 5/10
140/140 [==============================] - 1s 6ms/step - loss: 0.0177 - accuracy: 0.9957 - val_loss: 0.0700 - val_accuracy: 0.9740
Epoch 6/10
140/140 [==============================] - 1s 6ms/step - loss: 0.0148 - accuracy: 0.9966 - val_loss: 0.0703 - val_accuracy: 0.9731
Epoch 7/10
140/140 [==============================] - 1s 6ms/step - loss: 0.0123 - accuracy: 0.9973 - val_loss: 0.0708 - val_accuracy: 0.9731
Epoch 

In [ ]:
# Check the results
simple_dense_model.evaluate(X_val_sentences, y_val_labels)

35/35 [==============================] - 0s 3ms/step - loss: 0.0730 - accuracy: 0.9722


[0.07299549132585526, 0.9721972942352295]

In [ ]:
# Make prediction
simple_dense_model_pred_probs = simple_dense_model.predict(X_val_sentences)
simple_dense_model_pred_probs[:10]

array([[5.8724131e-03],
       [2.9376682e-03],
       [9.9785882e-01],
       [9.9585402e-01],
       [7.2453945e-04],
       [7.0282193e-03],
       [5.4465934e-05],
       [9.6232584e-04],
       [7.9644786e-04],
       [5.2953325e-04]], dtype=float32)

In [ ]:
# Turn pred_probs into single_dimension using tf.round()
simple_dense_model_preds = tf.squeeze(tf.round(simple_dense_model_pred_probs))
simple_dense_model_preds[:10]


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [ ]:
# Calculate the model results
simple_dense_model_results = calculate_results(y_true=y_val_labels,
                                               y_pred=simple_dense_model_preds)
simple_dense_model_results

{'accuracy': 97.21973094170404,
 'precision': 0.9718598228971536,
 'recall': 0.9721973094170404,
 'f1': 0.9715704842822593}

In [ ]:
# Create a helper function to compare our baseline results to new model results
def compare_baseline_to_new_results(baseline_results, new_model_results):
    for key, value in baseline_results.items():
        print(f"Baseline {key}: {value:.2f}, New {key}: {new_model_results[key]:.2f}, Difference: {new_model_results[key]-value:.2f}")

compare_baseline_to_new_results(baseline_results=baseline_model_results, 
                                new_model_results=simple_dense_model_results)

Baseline accuracy: 95.61, New accuracy: 97.22, Difference: 1.61
Baseline precision: 0.96, New precision: 0.97, Difference: 0.01
Baseline recall: 0.96, New recall: 0.97, Difference: 0.02
Baseline f1: 0.95, New f1: 0.97, Difference: 0.02


In [ ]:
# Combine model results into a DataFrame
all_model_results = pd.DataFrame({"baseline": baseline_model_results,
                                  "simple_dense": simple_dense_model_results,
                                 })
all_model_results = all_model_results.transpose()
all_model_results

,accuracy,precision,recall,f1
baseline,95.605381,0.958211,0.956054,0.952802
simple_dense,97.219731,0.971860,0.972197,0.971570


In [ ]:
# Get mean pred probs for 3 models
baseline_pred_probs = np.max(baseline_model.predict_proba(X_val_sentences), axis=1) # get the prediction probabilities from baseline model
combined_pred_probs = baseline_pred_probs + tf.squeeze(simple_dense_model_pred_probs, axis=1) 
combined_preds = tf.round(combined_pred_probs/2) # average and round the prediction probabilities to get prediction classes
combined_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 1., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [ ]:
# Calculate results from averaging the prediction probabilities
ensemble_results = calculate_results(y_val_labels, combined_preds)
ensemble_results

{'accuracy': 82.78026905829596,
 'precision': 0.9066884449666035,
 'recall': 0.8278026905829596,
 'f1': 0.8483708131366805}

In [ ]:
# Add our combined model's results to the results DataFrame
all_model_results.loc["ensemble_results"] = ensemble_results

In [ ]:
all_model_results

,accuracy,precision,recall,f1
baseline,95.605381,0.958211,0.956054,0.952802
simple_dense,97.219731,0.971860,0.972197,0.971570
ensemble_results,82.780269,0.906688,0.827803,0.848371


In [ ]:
# Finding the most wrong examples
# 1. Create dataframe with validation sentences and best performing model predictions
val_df = pd.DataFrame({"text": X_val_sentences,
                       "target": y_val_labels,
                       "pred": simple_dense_model,
                       "pred_prob": tf.squeeze(simple_dense_model_pred_probs)})
val_df.head()


,text,target,pred,pred_prob
0,Great comedy..cant stop laughing da:),0,<keras.engine.functional.Functional object at ...,0.005872
1,Its sarcasm.. .nt scarcasim,0,<keras.engine.functional.Functional object at ...,0.002938
2,We tried to contact you re your reply to our o...,1,<keras.engine.functional.Functional object at ...,0.997859
3,FreeMsg: Claim ur 250 SMS messages-Text OK to ...,1,<keras.engine.functional.Functional object at ...,0.995854
4,Ya that one is slow as poo,0,<keras.engine.functional.Functional object at ...,0.000725


In [ ]:
# Find the wrong predictions and sort by prediction probabilities
most_wrong = val_df[val_df["target"] != val_df["pred"]].sort_values("pred_prob", ascending=False)
most_wrong[:20]

,text,target,pred,pred_prob
1050,Congratulations ur awarded 500 of CD vouchers ...,1,<keras.engine.functional.Functional object at ...,0.999993
438,This message is free. Welcome to the new & imp...,1,<keras.engine.functional.Functional object at ...,0.999984
153,URGENT! Your Mobile number has been awarded wi...,1,<keras.engine.functional.Functional object at ...,0.999984
420,4mths half price Orange line rental & latest c...,1,<keras.engine.functional.Functional object at ...,0.999975
274,You have an important customer service announc...,1,<keras.engine.functional.Functional object at ...,0.999974
662,"URGENT! Your mobile was awarded a å£1,500 Bonu...",1,<keras.engine.functional.Functional object at ...,0.999966
681,XMAS iscoming & ur awarded either å£500 CD gif...,1,<keras.engine.functional.Functional object at ...,0.999963
58,Get ur 1st RINGTONE FREE NOW! Reply to this ms...,1,<keras.engine.functional.Functional object at ...,0.999955
1032,Urgent! call 09066350750 from your landline. Y...,1,<keras.engine.functional.Functional object at ...,0.999954
494,Urgent! Please call 09061743810 from landline....,1,<keras.engine.functional.Functional object at ...,0.999933


In [ ]:
# Chechking false positive(predicted: 1, should have been: 0)
for row in most_wrong[:10].itertuples(): # loop through the top 10 rows (change the index to view different rows)
    _, text, target, pred, prob = row
    print(f"Target: {target}, Pred: {pred}, Prob: {prob}")
    print(f"Text:\n{text}\n")
    print("----\n")

Target: 1, Pred: <keras.engine.functional.Functional object at 0x7f2058024be0>, Prob: 0.9999927282333374
Text:
Congratulations ur awarded 500 of CD vouchers or 125gift guaranteed & Free entry 2 100 wkly draw txt MUSIC to 87066 TnCs www.Ldew.com1win150ppmx3age16

----

Target: 1, Pred: <keras.engine.functional.Functional object at 0x7f2058024be0>, Prob: 0.9999843835830688
Text:
This message is free. Welcome to the new & improved Sex & Dogging club! To unsubscribe from this service reply STOP. msgs@150p 18+only

----

Target: 1, Pred: <keras.engine.functional.Functional object at 0x7f2058024be0>, Prob: 0.999983549118042
Text:
URGENT! Your Mobile number has been awarded with a å£2000 prize GUARANTEED. Call 09058094454 from land line. Claim 3030. Valid 12hrs only

----

Target: 1, Pred: <keras.engine.functional.Functional object at 0x7f2058024be0>, Prob: 0.9999750852584839
Text:
4mths half price Orange line rental & latest camera phones 4 FREE. Had your phone 11mths ? Call MobilesDirect fr

In [ ]:
# Checking false negative(predicted: 0, should have been: 1)
for row in most_wrong[-10:].itertuples():
    _, text, target, pred, prob = row
    print(f"Target: {target}, Pred: {pred}, Prob: {prob}")
    print(f"Text:\n{text}\n")
    print("----\n")

Target: 0, Pred: <keras.engine.functional.Functional object at 0x7f2058024be0>, Prob: 2.1128521439095493e-06
Text:
Hey, can you tell me blake's address? Carlos wanted me to meet him there but I got lost and he's not answering his phone

----

Target: 0, Pred: <keras.engine.functional.Functional object at 0x7f2058024be0>, Prob: 2.0981442503398284e-06
Text:
Thanks honey but still haven't heard anything I will leave it a bit longer so not 2 crowd him and will try later - great advice thanks hope cardiff is still there!

----

Target: 0, Pred: <keras.engine.functional.Functional object at 0x7f2058024be0>, Prob: 1.977963165700203e-06
Text:
Yar lor actually we quite fast... Cos da ge slow wat... Haha...

----

Target: 0, Pred: <keras.engine.functional.Functional object at 0x7f2058024be0>, Prob: 1.8490298998585786e-06
Text:
I'm stuck in da middle of da row on da right hand side of da lt... 

----

Target: 0, Pred: <keras.engine.functional.Functional object at 0x7f2058024be0>, Prob: 1.75408570

It seems that when we check the false positive and false negative examples for checking, our model seems to be very successful.

In [ ]:
# Let's check 
sample_1 = "Congratulations! You've won a $1000 Walmart gift card. Go to http://bit.ly/123456 tp claim now"
sample_2 = "Did you get the message I sent you yesterday about the car price? Please let me know if you got it. "

In [ ]:
def predict_on_sentence(model, sentence):
    pred_prob = model.predict([sentence])
    pred_label = tf.squeeze(tf.round(pred_prob).numpy())
    print(f"Pred: {pred_label}", "(spam)" if pred_label > 0 else "not spam", f"Prob: {pred_prob[0][0]}")
    print(f"Text:\n{sentence}")

In [ ]:
# Make a prediction on sample_1
predict_on_sentence(model=simple_dense_model, 
                    sentence=sample_1)

Pred: 1.0 (spam) Prob: 0.9724246859550476
Text:
Congratulations! You've won a $1000 Walmart gift card. Go to http://bit.ly/123456 tp claim now


In [ ]:
predict_on_sentence(model=simple_dense_model, 
                    sentence=sample_2)

Pred: 0.0 not spam Prob: 0.00229619606398046
Text:
Did you get the message I sent you yesterday about the car price? Please let me know if you got it. 


In [ ]:
simple_dense_model.save('spam_detection_model')


2023-02-24 21:36:55.955611: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: spam_detection_model/assets


In [ ]:
spam_detection_model = tf.keras.models.load_model('spam_detection_model')


In [ ]:
predict_on_sentence(model=spam_detection_model, 
                    sentence=sample_2)

Pred: 0.0 not spam Prob: 0.00229619606398046
Text:
Did you get the message I sent you yesterday about the car price? Please let me know if you got it. 
